In [19]:
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("EcommerceDataset_Final_2.csv").getOrCreate()

df = spark.read.csv('EcommerceDataset_Final_2.csv', header=True, inferSchema=True)
df.show()


+-------+-------------------+-------+--------+---------+-------------------+------------+----------+---------+----------+-------------------+-------------------+----------+--------------------------+--------------------+--------------------+--------------------+--------------+-------------+----------+
|OrderID|       CustomerName|Product|Quantity|UnitPrice|          OrderDate|ShippingCost|CustomerID|  Country|Membership|         SignUpDate|      LastOrderDate|TotalSpent|Customer Communication Log|   CommunicationDate|         InquiryType|  CommunicationIssue| EmailUsername|  EmailDomain|OrderValue|
+-------+-------------------+-------+--------+---------+-------------------+------------+----------+---------+----------+-------------------+-------------------+----------+--------------------------+--------------------+--------------------+--------------------+--------------+-------------+----------+
|    614|     Patricia Smith|   Book|       2|   330.71|2024-09-05 00:00:00|       36.96|  

In [ ]:
# #Count: 975
# Mean: 1259.67
# Standard Deviation: 948.23
# Minimum: 10.16
# 25th Percentile: 499.93
# Median (50th Percentile): 999.86
# 75th Percentile: 1873.57
# Maximum: 3998.44

In [20]:
from pyspark.sql.functions import *
threshold = 499.93  # 25th percentile value

# Create a new column 'Churn' based on the threshold
df = df.withColumn("Churn", when(col("OrderValue") < threshold, 1).otherwise(0))
df.show()

+-------+-------------------+-------+--------+---------+-------------------+------------+----------+---------+----------+-------------------+-------------------+----------+--------------------------+--------------------+--------------------+--------------------+--------------+-------------+----------+-----+
|OrderID|       CustomerName|Product|Quantity|UnitPrice|          OrderDate|ShippingCost|CustomerID|  Country|Membership|         SignUpDate|      LastOrderDate|TotalSpent|Customer Communication Log|   CommunicationDate|         InquiryType|  CommunicationIssue| EmailUsername|  EmailDomain|OrderValue|Churn|
+-------+-------------------+-------+--------+---------+-------------------+------------+----------+---------+----------+-------------------+-------------------+----------+--------------------------+--------------------+--------------------+--------------------+--------------+-------------+----------+-----+
|    614|     Patricia Smith|   Book|       2|   330.71|2024-09-05 00:00:

In [21]:
# Group by CustomerID and calculate churn rate for each customer
customer_churn = df.groupBy("CustomerID").agg((sum(col("Churn")) / count(col("CustomerID"))).alias("ChurnRate"))

# Show the updated DataFrame with the new 'ChurnRate' column for each customer
customer_churn.show()

+----------+------------------+
|CustomerID|         ChurnRate|
+----------+------------------+
|       463|               1.0|
|       471|               0.0|
|       496|               0.0|
|       392|               0.0|
|       540|               1.0|
|       623|               0.0|
|       737|               0.5|
|        31|               0.0|
|       516|               0.5|
|        85|               1.0|
|       137|               1.0|
|       451|               0.0|
|        65|               0.0|
|       458|               1.0|
|        53|               0.0|
|       255|               0.0|
|       481|               0.5|
|       588|               0.0|
|       799|               0.5|
|       133|0.3333333333333333|
+----------+------------------+
only showing top 20 rows



In [22]:
# Calculate retention based on 'CustomerID'

# Define the observation period (e.g., one year from the latest order date) 
observation_period_end = df.agg({"OrderDate": "max"}).collect()[0][0]
observation_period_end



'2025-09-26 00:00:00'

In [23]:
df = df.withColumn('Retention', col('OrderDate') >= (lit(observation_period_end) - expr('INTERVAL 1 YEAR')))
df.show()

+-------+-------------------+-------+--------+---------+-------------------+------------+----------+---------+----------+-------------------+-------------------+----------+--------------------------+--------------------+--------------------+--------------------+--------------+-------------+----------+-----+---------+
|OrderID|       CustomerName|Product|Quantity|UnitPrice|          OrderDate|ShippingCost|CustomerID|  Country|Membership|         SignUpDate|      LastOrderDate|TotalSpent|Customer Communication Log|   CommunicationDate|         InquiryType|  CommunicationIssue| EmailUsername|  EmailDomain|OrderValue|Churn|Retention|
+-------+-------------------+-------+--------+---------+-------------------+------------+----------+---------+----------+-------------------+-------------------+----------+--------------------------+--------------------+--------------------+--------------------+--------------+-------------+----------+-----+---------+
|    614|     Patricia Smith|   Book|      

In [28]:
# Calculate retention rate for each customer
retention_rate = df.groupBy('CustomerID').agg((count(when(col('Retention') == True, 1)) / count('*')).alias('RetentionRate'))
# Merge the retention rate
df = df.join(retention_rate, on='CustomerID', how='left')
df=df.sort("CustomerID",asending=True)
df.show()

2024-08-21 17:21:28,557 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+-------+-------------------+-------+--------+---------+-------------------+------------+---------+----------+-------------------+-------------------+----------+--------------------------+--------------------+--------------------+--------------------+--------------+-------------+----------+-----+---------+-------------+-------------+-------------+-------------+-------------+
|CustomerID|OrderID|       CustomerName|Product|Quantity|UnitPrice|          OrderDate|ShippingCost|  Country|Membership|         SignUpDate|      LastOrderDate|TotalSpent|Customer Communication Log|   CommunicationDate|         InquiryType|  CommunicationIssue| EmailUsername|  EmailDomain|OrderValue|Churn|Retention|RetentionRate|RetentionRate|RetentionRate|RetentionRate|RetentionRate|
+----------+-------+-------------------+-------+--------+---------+-------------------+------------+---------+----------+-------------------+-------------------+----------+--------------------------+--------------------+--